In [ ]:
%load_ext autoreload
%autoreload 2

%config IPCompleter.greedy=True

# $r_\text{max}$ vs. $v_\text{max}$

... where $r_\text{max}$ is the radius of maximal circular velocity.

First, imports:

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import h5py
from astropy import units
from pathlib import Path
import os

import snapshot_obj
import dataset_compute
import curve_fit

import importlib

In [ ]:
importlib.reload(snapshot_obj)
importlib.reload(dataset_compute)
importlib.reload(curve_fit)

## Construct data dictionary

Add entries for each simulation, and specify M31 and MW galaxies:

In [ ]:
snap_id = 127
sim_ids = ["V1_LR_fix", "V1_LR_curvaton_p082_fix", "V1_LR_curvaton_p084_fix"]
names = ["LCDM", "p082", "p084"]

# Define M31 and MW in each simulation:
m31 = [(1,0), (1,0), (1,0)]
mw = [(2,0), (1,1), (1,0)]

data = {}
for name, sim_id, m31_ns, mw_ns in zip(names, sim_ids, m31, mw):
    data[name] = {"snapshot": snapshot_obj.Snapshot(sim_id, snap_id, name=name),
                  "M31_identifier": m31_ns,
                  "MW_identifier": mw_ns}

In [ ]:
snap_id = 126
sim_ids = ["V1_MR_fix", "V1_MR_curvaton_p082_fix"]
names = ["LCDM", "p082"]

# Define M31 and MW in each simulation:
m31 = [(1,0), (1,0)]
mw = [(2,0), (1,1)]

data = {}
for name, sim_id, m31_ns, mw_ns in zip(names, sim_ids, m31, mw):
    data[name] = {"snapshot": snapshot_obj.Snapshot(sim_id, snap_id, name=name),
                  "M31_identifier": m31_ns,
                  "MW_identifier": mw_ns}

Choose how to distinguish between satellite and isolated galaxies:

In [ ]:
distinction = "by_r"
lum_or_dark = "dark"

Read datasets, split into satellites and isolated galaxies, and add to the data dictionary. We also disregard dark halos and potential spurious halos with $v_\mathrm{max} = 0$.

In [ ]:
for name, sim_data in data.items():
    # Get data:
    snap = sim_data["snapshot"]
    max_point = snap.get_subhalos("Max_Vcirc", "Extended")
    vmax = max_point[:,0] * units.cm.to(units.km)
    rmax = max_point[:,1] * units.cm.to(units.kpc)
    
    # Split into satellites:
    if distinction == "by_r":
        masks_sat, mask_isol = dataset_compute.split_satellites_by_distance(
            snap, sim_data["M31_identifier"], sim_data["MW_identifier"])
    elif distinction == "by_gn":
        masks_sat, mask_isol = dataset_compute.split_satellites_by_group_number(
            snap, sim_data["M31_identifier"], sim_data["MW_identifier"])
        
    mask_lum, mask_dark = dataset_compute.split_luminous(snap)
    mask_nonzero_vmax = dataset_compute.prune_vmax(snap)

    # Add to dictionary:
    for dataset_name, dataset in zip(["Vmax", "Rmax"],[vmax, rmax]):
        data[name][dataset_name] = \
        {"satellites": 
         {"luminous": dataset[np.logical_and.reduce(
             [np.logical_or.reduce(masks_sat), mask_lum, mask_nonzero_vmax]
         )],
          "dark": dataset[np.logical_and.reduce(
              [np.logical_or.reduce(masks_sat), mask_dark, mask_nonzero_vmax]
          )]
         },
         "isolated":
         {"luminous": dataset[np.logical_and.reduce(
             [mask_isol, mask_lum, mask_nonzero_vmax]
         )],
          "dark": dataset[np.logical_and.reduce(
              [mask_isol, mask_dark, mask_nonzero_vmax]
          )]
         }
        }

## Plot

Plot satellites and isolated galaxies on different subplots and add median fits for each dataset.

In [ ]:
# Set some parameters:
x_down = 10; x_up = 100
y_down = 0.1; y_up = 100

# Set marker styles:
fcolor = ["black", "red", "blue", "green"]
mcolor = ["gray", "pink", "lightblue", "lightgreen"]
marker = ['+', "o", "^", 1]

In [ ]:
# Construct saving location:
if lum_or_dark == "luminous":
    filename = 'Rmax_vs_Vmax_lum_{}'.format(distinction)
else:
    filename = 'Rmax_vs_Vmax_dark_{}'.format(distinction)
for name in names:
    filename += "_{}".format(name)
filename += ".png"
    
home = os.path.dirname(snapshot_obj.__file__)
path = os.path.join(home,"Figures", "MediumResolution")
filename = os.path.join(path, filename)

In [ ]:
fig, axes = plt.subplots(ncols=2, figsize=(14,6))
plt.subplots_adjust(wspace=0.3)

# Set axis:
for ax in axes:
    ax.set_xscale('log')
    ax.set_yscale('log')
    ax.set_xlim(x_down, x_up)
    ax.set_ylim(y_down, y_up)    
    ax.set_xlabel('$v_{\mathrm{max}}[\mathrm{km s^{-1}}]$', fontsize=16)
    ax.set_ylabel('$r_{\mathrm{max}}[\mathrm{kpc}]$', fontsize=16)

axes[0].set_title('Satellites')
axes[1].set_title('Isolated galaxies')

# Add scatter plots:
for i, (name, entry) in enumerate(data.items()):
    x = entry['Vmax']['satellites'][lum_or_dark]
    y = entry['Rmax']['satellites'][lum_or_dark]
    axes[0].scatter(x, y, s=20, marker=marker[i], c=mcolor[i], \
        edgecolor='none', label=name)
    
    x = entry['Vmax']['isolated'][lum_or_dark]
    y = entry['Rmax']['isolated'][lum_or_dark]
    axes[1].scatter(x, y, s=20, marker=marker[i], c=mcolor[i], \
        edgecolor='none', label=name)
    
# Add median curves:
n_median_points = 7
for i, (name, entry) in enumerate(data.items()):
    x = entry['Vmax']['satellites'][lum_or_dark]
    y = entry['Rmax']['satellites'][lum_or_dark]
    print("# of satellites: {}".format(x.size))
    median = curve_fit.median_trend(x, y, n_points_per_bar=n_median_points)
    if median is not None:
        axes[0].plot(median[0], median[1], c=fcolor[i], linestyle='--')
    else:
        print("Could not fit median for:", name)
    
    x = entry['Vmax']['isolated'][lum_or_dark]
    y = entry['Rmax']['isolated'][lum_or_dark]
    print("# of isolated galaxies: {}".format(x.size))
    median = curve_fit.median_trend(x, y, n_points_per_bar=n_median_points)
    if median is not None:
        axes[1].plot(median[0], median[1], c=fcolor[i], linestyle='--')
    else:
        print("Could not fit median for:", name)
    
axes[0].legend(loc='lower right')
plt.tight_layout()

plt.savefig(filename, dpi=200)